In [1]:
import numpy as np
import tensorflow as tf
import keras
import itertools
import matplotlib.pyplot as plt

from keras import backend as k

from keras.models import Sequential, Model, model_from_json

from keras.layers import Input, Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, Reshape, LSTM, Bidirectional
from keras.layers.convolutional import *
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense,Flatten,Dropout, Activation
from keras.layers.normalization import BatchNormalization

from keras.preprocessing import image 
from keras.preprocessing.image import ImageDataGenerator

from keras.metrics import categorical_crossentropy
from keras.optimizers import SGD, Adam

from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from keras.models import load_model

In [2]:
train_path='../input/ekushdb/train/train'
test_path= '../input/ekushdb/test/test'

def myFunc(image):
    image = np.array(image)
    image= np.expand_dims(image, axis=0)
    image = image.astype('float32')
    image /= 255
    return image
train_datagen = ImageDataGenerator(preprocessing_function=myFunc, validation_split=0.3) #set validation split


train_batches= train_datagen.flow_from_directory(train_path,target_size=(64,64),color_mode='grayscale',batch_size=64,subset='training')
valid_batches= train_datagen.flow_from_directory(train_path,target_size=(64,64),color_mode='grayscale',batch_size=64,subset='validation')
test_batches= ImageDataGenerator(preprocessing_function=myFunc).flow_from_directory(test_path,target_size=(64,64),color_mode='grayscale',batch_size=64,shuffle=False)

model = Sequential()

#parameters
img_w, img_h = 64, 64

#model
input_shape = (img_w, img_h, 1)
inputs = Input(shape=input_shape, dtype='float32')


inner = Conv2D(16, (5, 5), activation='relu', padding='same', name='conv1', kernel_initializer='he_normal')(inputs)
inner = Conv2D(32, (5, 5), activation='relu', padding='same', name='conv2', kernel_initializer='he_normal')(inner)

inner = MaxPooling2D(pool_size=(2, 2), name='max1')(inner)
inner = Dropout((0.5), name='drop1')(inner)
inner = BatchNormalization()(inner)


inner = Conv2D(64, (5, 5), activation='relu', padding='same', name='conv3', kernel_initializer='he_normal')(inner)
inner = Conv2D(128, (5, 5), activation='relu', padding='same', name='conv4', kernel_initializer='he_normal')(inner)

inner = MaxPooling2D(pool_size=(2, 2), name='max2')(inner)
inner = Dropout((0.5), name='drop2')(inner)
inner = BatchNormalization()(inner)

#cnn to lstm
x = 16 * 16
inner = Reshape(target_shape=(x,128))(inner) 

#LSTM layers
inner = Bidirectional(LSTM(256, name='lstm1', return_sequences=True))(inner)

inner = Bidirectional(LSTM(128, name='lstm2'))(inner)


inner = Dense(512, activation='relu', kernel_initializer='he_normal', name='dense1')(inner)
inner = (Dropout(0.5))(inner)
inner = Dense(122, activation='softmax', name='dense2')(inner)

model= Model(inputs = inputs, outputs = inner)

checkpoint = keras.callbacks.ModelCheckpoint('/kaggle/working/model-{epoch:03d}-{accuracy:03f}-{val_accuracy:03f}.h5', verbose=1, monitor='val_accuracy',save_best_only=True, mode='max')

#model = load_model('../input/main-model-2/model-046-0.995101-0.970464.h5')

model.compile(Adam(lr=0.0005),loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

Found 179189 images belonging to 122 classes.
Found 76716 images belonging to 122 classes.
Found 110126 images belonging to 122 classes.
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 16)        416       
_________________________________________________________________
conv2 (Conv2D)               (None, 64, 64, 32)        12832     
_________________________________________________________________
max1 (MaxPooling2D)          (None, 32, 32, 32)        0         
_________________________________________________________________
drop1 (Dropout)              (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)  

In [3]:
his = model.fit_generator(train_batches, epochs =50, verbose=1,validation_data = valid_batches,callbacks = [checkpoint])

Epoch 1/50
2800/2800 [==============================] - ETA: 0s - loss: 1.7061 - accuracy: 0.5471
Epoch 00001: val_accuracy improved from -inf to 0.83500, saving model to /kaggle/working/model-001-0.547132-0.835002.h5
2800/2800 [==============================] - 423s 151ms/step - loss: 1.7061 - accuracy: 0.5471 - val_loss: 0.5782 - val_accuracy: 0.8350
Epoch 2/50
2800/2800 [==============================] - ETA: 0s - loss: 0.5444 - accuracy: 0.8504
Epoch 00002: val_accuracy improved from 0.83500 to 0.88202, saving model to /kaggle/working/model-002-0.850443-0.882019.h5
2800/2800 [==============================] - 388s 139ms/step - loss: 0.5444 - accuracy: 0.8504 - val_loss: 0.4184 - val_accuracy: 0.8820
Epoch 3/50
2800/2800 [==============================] - ETA: 0s - loss: 0.3667 - accuracy: 0.9015
Epoch 00003: val_accuracy improved from 0.88202 to 0.93074, saving model to /kaggle/working/model-003-0.901512-0.930745.h5
2800/2800 [==============================] - 389s 139ms/step - los

In [4]:
loss, acc = model.evaluate_generator(test_batches, verbose=1)
print(acc)
print(loss)

1721/1721 [==============================] - 237s 138ms/step - loss: 0.4953 - accuracy: 0.9480
0.9480050206184387
0.4953276813030243


In [5]:
model.save('/kaggle/working/fmodel.h5')